In [ ]:
#Area calc -> CRS:6875

# from osgeo import gdal, ogr, osr
import numpy as np
import cv2
import requests

import rasterio
import geopandas as gpd
from rasterio.features import shapes
from shapely.geometry import shape, box

from owslib.wms import WebMapService
import pandas as pd

from PIL import Image
import io

In [ ]:

min_lat = 44.5084337
min_lon = 11.3444083
max_lat = 44.5090871
max_lon = 11.3456463

# min_lat = 42.2005615
# min_lon = 13.6668152
# max_lat = 42.2008607
# max_lon = 13.6670757

# #Super piccolo fabbricato
# min_lat = 44.5091362
# min_lon = 11.3454735
# max_lat = 44.5091508
# max_lon = 11.3454969


# Definizione dei colori
color_particella = np.array([189, 236, 253])  # CP.CadastralParcel
color_fabbricato = np.array([19, 128, 236])  # Fabbricati

# Percorso dell'immagine
input_excel_path = r"IO\Input\input_bbox.xlsx"
output_shapefile = r'IO\Output\shape_estratte.shp'


## Codice per WMS

In [ ]:
#Area calc -> CRS:6875

# from osgeo import gdal, ogr, osr
import numpy as np
import cv2
import requests

import rasterio
import geopandas as gpd
from rasterio.features import shapes
from shapely.geometry import shape, box

from owslib.wms import WebMapService
import pandas as pd

from PIL import Image
import io

def SmoothShape(gdf: gpd.GeoDataFrame):

  '''
  Return a gdf with smoothed geometry.
  '''
  
  #SMOOTHING delle shape filtrate
  # Set tolerance for shape smoothing
  tolerance: float = 0.000001

  gdfs_merged['geometry'] = gdfs_merged['geometry'].simplify(tolerance, preserve_topology=True)

def RequestFeatureOnPoint(row):

  try:
      response = session.get(f'https://wms.cartografia.agenziaentrate.gov.it/inspire/ajax/ajax.php?op=getDatiOggetto&lon={row["shape_centroid"].x}&lat={row["shape_centroid"].y}')
      lista_features_response.append(response.json())
  except Exception as e:
      print(f"Errore nella richiesta di Feature: {e}")

def ScaleUpBox(df : pd.DataFrame) -> None:

  '''
  Scala le coordinate in base alla dimensione del bbox (in modo da poter eliminare le eventuali shape vicine utilizzando il bbox + soglia)
  '''
  small_bbox_threshold: float = 0.00001

  scale_value_small: float = 0.00007
  scale_value_normal: float = 0.00003

  #Non è area in 2d, ma una approssimazione su lon e lat
  df["bbox_size"] = (df["max_lat"] - df["min_lat"]) * (df["max_lon"] - df["min_lon"])

  #Se il bbox è small viene scalato più del bbox normal
  df["scale_value"] = np.where(df["bbox_size"] <= small_bbox_threshold, scale_value_small, scale_value_normal)

  df["min_lat_scaled"] = df["min_lat"] - df["scale_value"]
  df["min_lon_scaled"] = df["min_lon"] - df["scale_value"]
  df["max_lat_scaled"] = df["max_lat"] + df["scale_value"]
  df["max_lon_scaled"] = df["max_lon"] + df["scale_value"]

def ExtractShapeFromRow(row):
  def GetShapesFromWMS(row):
    try:
      img =  wms.getmap(layers=["CP.CadastralParcel", "fabbricati"],
                        srs="EPSG:6706",
                        bbox=(row["min_lat_scaled"], row["min_lon_scaled"], row["max_lat_scaled"], row["max_lon_scaled"]),
                        size=(2048, 2048),
                        format="image/png",
                        transparent=True
                        )
      img_data = img.read()
      img_pil = Image.open(io.BytesIO(img_data))
      img_cv2 = np.array(img_pil)
      img_cv2 = cv2.cvtColor(img_cv2, cv2.COLOR_RGB2BGR)

      return img_cv2
    
    except:
      raise ValueError("Errore nella chiamata al WMS")

  def ComputeRaster(row):
    # Carica l'immagine con OpenCV (BGR format)
    img = GetShapesFromWMS(row)

    # Crea maschere per i due colori
    mask_fabbricato = cv2.inRange(img, color_fabbricato, color_fabbricato)
    mask_particella = cv2.inRange(img, color_particella, color_particella)

    # Carica il raster con rasterio
    with rasterio.MemoryFile() as temp_file:
        with temp_file.open(driver="PNG", width=2048, height=2048, count=3, dtype="uint8") as src:  #count=3 RGB band
          
          src.write(img.transpose(2,0,1)) #traspose per input compliant con rasterio

          # Trasformazione tra bounding box e pixel
          transform = rasterio.transform.from_bounds(row["min_lon_scaled"], row["min_lat_scaled"], row["max_lon_scaled"], row["max_lat_scaled"], 2048, 2048)

          # Polygonizza la maschera arancione
          shapes_fabbricato = shapes(mask_fabbricato, mask=mask_fabbricato > 0, transform=transform)
          
          # Polygonizza la maschera gialla
          shapes_particella = shapes(mask_particella, mask=mask_particella > 0, transform=transform)

          # Converti entrambe le maschere in GeoDataFrame e assegna le proprietà "Fabbricato" e "Particella"
          geoms_fabbricati = [{'properties': {'tipo': 'Fabbricato'}, 'geometry': shape(geom)} for geom, value in shapes_fabbricato]
          geoms_particelle = [{'properties': {'tipo': 'Particella'}, 'geometry': shape(geom)} for geom, value in shapes_particella]
          
          # Combina i geodataframe
          gdf = gpd.GeoDataFrame.from_features(geoms_fabbricati + geoms_particelle, crs=6706)

    # ** Elimina le shape vicine ai bordi del bbox **
    # Crea il bounding box e riducilo usando la soglia di distanza
    bbox = box(row["min_lon_scaled"], row["min_lat_scaled"], row["max_lon_scaled"], row["max_lat_scaled"])
    bbox_distance_threshold: float = 0.00002
    bbox_restricted = bbox.buffer(-bbox_distance_threshold)

    # Filtra le geometrie all'interno del bbox ridotto
    gdf = gdf[gdf.geometry.within(bbox_restricted)].reset_index(drop=True)
    
    lista_gdfs_extracted_shapes_in_bbox.append(gdf)



  ComputeRaster(row)


# MAIN


wms = WebMapService('https://wms.cartografia.agenziaentrate.gov.it/inspire/wms/ows01.php?', version='1.3.0')

session = requests.session()
lista_features_response = []
lista_gdfs_extracted_shapes_in_bbox = []

# Definizione dei colori
color_particella = np.array([189, 236, 253])  # CP.CadastralParcel
color_fabbricato = np.array([19, 128, 236])  # Fabbricati

# Percorso dell'immagine
input_excel_path = r"IO\Input\input_bbox.xlsx"
output_shapefile = r'IO\Output\shape_estratte.shp'

df = pd.read_excel(input_excel_path,
                  usecols=["id", "cod_comune", "sezione", "foglio", "allegato", "sviluppo", "particella", "min_lat", "min_lon", "max_lat", "max_lon"],
                  dtype={"id" : str, "cod_comune" : str, "sezione" : str, "foglio" : str, "allegato" : str, "sviluppo" : str, "particella" : str, "min_lat" : float, "min_lon" : float, "max_lat" : float, "max_lon" : float})

ScaleUpBox(df)

df.apply(ExtractShapeFromRow, axis=1)

gdf_extracted_shapes_in_bbox = pd.concat(lista_gdfs_extracted_shapes_in_bbox)

ComputeGdfAreaAndCentroid

gdf_to_reproject.apply(RequestFeatureOnPoint, axis=1)

gdf_merged = pd.concat([gdf_to_reproject, pd.DataFrame(lista_features_response)], axis=1)


#Escludo tutte le shape con features non corrispondenti all'input
df_filtered = df.merge(gdf_merged, how="left", left_on=["cod_comune", "foglio", "particella"], right_on=["COD_COMUNE", "FOGLIO", "NUM_PART"])

gdf_filtered = gpd.GeoDataFrame(df_filtered, geometry="geometry", crs=4326)

SmoothShape()


# Salva il risultato come shapefile
gdf_filtered.pop("shape_centroid")
gdf_filtered.to_file(output_shapefile)

print("end")